In [5]:
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

data = pd.read_csv('../data/preprocessed/engineered_data.csv')

# # Round mood values to discrete values of 1-10
# data['mood'] = data['mood'].round().astype(int)
# data['mood'] = data['mood'].clip(lower=1, upper=10)

data = data.sort_values(by=['id', 'day'])
groups = data['id']

gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Split the dataset
for train_idx, test_idx in gss.split(data, groups=groups):
    train_set = data.iloc[train_idx]
    test_set = data.iloc[test_idx]

train_set.to_csv('../data/preprocessed/train_set.csv', index=False)
test_set.to_csv('../data/preprocessed/test_set.csv', index=False)
